In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

from env import user, host, password 
import wrangle
import split_scale
import features

##### As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of $5.00 per customer.

## select your features using your features script


In [ ]:
#We'll need to get the data. I think it's the telco_churn stuff. I think I have a home-made
#module for that.


In [2]:
df = wrangle.wrangle_telco()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1685 entries, 0 to 1694
Data columns (total 4 columns):
customer_id        1685 non-null object
total_charges      1685 non-null float64
monthly_charges    1685 non-null float64
tenure             1685 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 65.8+ KB


In [4]:
train,test=split_scale.split_my_data(df)

In [6]:
train = train.set_index('customer_id') 
test = test.set_index('customer_id')

In [7]:
scaler,train_scaled, test_scaled =split_scale.standard_scaler(train,test)

In [8]:
X_train=train.drop(columns=['total_charges'])
X_test=test.drop(columns=['total_charges'])
y_train=train[['total_charges']]
y_test=test[['total_charges']]

In [ ]:
# def make_feature_matrixANDtarget(train, test, dep_var):
#     X_train=train.drop(columns=[dep_var])
#     X_test=test.drop(columns=[dep_var])
#     y_train=train[[dep_var]]
#     y_test=test[[dep_var]]
#     return X_train, X_test, y_train, y_test

#X_train, X_test, y_train, y_test = make_feature_matrixANDtarget(train, test, 'total_charges')

In [9]:
X_train.shape

(1348, 2)

In [10]:
X_test.shape

(337, 2)

In [11]:
y_train.shape

(1348, 1)

In [12]:
y_train.shape

(1348, 1)

In [15]:
optimal_n_features = features.optimal_number_of_features(X_train, y_train, X_test, y_test)

In [16]:
selected_features=features.optimal_features(X_train,y_train,optimal_n_features)

In [17]:
#Looks like tenure is not that important
selected_features

Index(['monthly_charges'], dtype='object')

## START THE SKLEARN MODEL 3-STEP 
    #####1 Call down the model from the SKLEARN world of platonic forms
    #####2 Fit the training data to the model (X and Y)
    #####3 Make some predictions or transformations with this fitted model

In [21]:
#INVOKE THE MODEL
lin_model_1=LinearRegression()

In [33]:
X_train.head()

,monthly_charges,tenure
2192-CKRLV,-0.349191,0.839851
7596-IIWYC,-1.184789,-1.700684
6408-WHTEF,0.811120,0.839851
3070-FNFZQ,-1.180459,-1.926509
8774-GSBUN,-1.036142,0.839851


In [22]:
#Fit that model to the training data
lin_model_1.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [26]:
#Looks like there is a stronger 
lin_model_1_y_intercept=lin_model_1.intercept_
print("intercept: ",lin_model_1_y_intercept)
lin_model_1_coefficents=lin_model_1.coef_
print("coefficents: ",lin_model_1_coefficents)

intercept:  [-1.25229225e-16]
coefficents:  [[0.82238767 0.29099306]]


In [30]:
y_predicted=lin_model_1.predict(X_test)

In [31]:
type(y_predicted)

numpy.ndarray

In [34]:
mse_lm1=mean_squared_error(y_test,y_predicted)
print("linear model \n mean_squared error: {:.3}".format(mse_lm1))

r2_lm1=r2_score(y_test,y_predicted)
print('{:.2%} of the variance in the student''s final grade can be explained by monthly charges and tenure'.format(r2_lm1))


linear model 
 mean_squared error: 0.0343
96.55% of the variance in the students final grade can be explained by monthly charges and tenure
